In [ ]:
# !python /home/ashish/llama_inference/clinical-outcome-prediction/tasks/mp/mp.py --mimic_dir '/home/ashish/llama_inference/physionet.org/files/mimiciii/1.4'  --save_dir '/home/ashish/llama_inference/clinical-outcome-prediction/saved_files'  --admission_only True \   

Traceback (most recent call last):
  File "/home/ashish/llama_inference/clinical-outcome-prediction/tasks/mp/mp.py", line 7, in <module>
    from tasks import mimic_utils
ModuleNotFoundError: No module named 'tasks'


In [1]:
cd clinical-outcome-prediction/

/home/ashish/llama_inference/clinical-outcome-prediction


/home/ashish/llama_inference/llama_env/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:

import os
import sys

from typing import List

from tasks import mimic_utils
import pandas as pd

In [3]:
task_name = "DIA_GROUPS_3_DIGITS_adm"


In [4]:
# if admission_only:
#     task_name = f"{task_name}_adm"

In [5]:
mimic_diagnoses = pd.read_csv(os.path.join('/home/ashish/llama_inference/physionet.org/files/mimiciii/1.4', "DIAGNOSES_ICD.csv"))


In [6]:
mimic_notes = pd.read_csv(os.path.join('/home/ashish/llama_inference/physionet.org/files/mimiciii/1.4', "NOTEEVENTS.csv"))
mimic_notes.info()

/tmp/ipykernel_2655931/2055088135.py:1: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  mimic_notes = pd.read_csv(os.path.join('/home/ashish/llama_inference/physionet.org/files/mimiciii/1.4', "NOTEEVENTS.csv"))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2083180 entries, 0 to 2083179
Data columns (total 11 columns):
 #   Column       Dtype  
---  ------       -----  
 0   ROW_ID       int64  
 1   SUBJECT_ID   int64  
 2   HADM_ID      float64
 3   CHARTDATE    object 
 4   CHARTTIME    object 
 5   STORETIME    object 
 6   CATEGORY     object 
 7   DESCRIPTION  object 
 8   CGID         float64
 9   ISERROR      float64
 10  TEXT         object 
dtypes: float64(3), int64(2), object(6)
memory usage: 174.8+ MB


In [7]:
mimic_admissions = pd.read_csv(os.path.join('/home/ashish/llama_inference/physionet.org/files/mimiciii/1.4', "ADMISSIONS.csv"))

In [8]:
# mimic_notes = mimic_utils.filter_notes(
#     mimic_notes, mimic_admissions, admission_text_only=True)

In [9]:
# mimic_notes

In [10]:
mimic_notes

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...
...,...,...,...,...,...,...,...,...,...,...,...
2083175,2070657,31097,115637.0,2132-01-21,2132-01-21 03:27:00,2132-01-21 03:38:00,Nursing/other,Report,17581.0,NaN,NPN\n\n\n#1 Infant remains in RA with O2 sats...
2083176,2070658,31097,115637.0,2132-01-21,2132-01-21 09:50:00,2132-01-21 09:53:00,Nursing/other,Report,19211.0,NaN,"Neonatology\nDOL #5, CGA 36 weeks.\n\nCVR: Con..."
2083177,2070659,31097,115637.0,2132-01-21,2132-01-21 16:42:00,2132-01-21 16:44:00,Nursing/other,Report,20104.0,NaN,Family Meeting Note\nFamily meeting held with ...
2083178,2070660,31097,115637.0,2132-01-21,2132-01-21 18:05:00,2132-01-21 18:16:00,Nursing/other,Report,16023.0,NaN,NPN 1800\n\n\n#1 Resp: [**Known lastname 2243*...


In [11]:
admissions_df = mimic_admissions

notes_df = mimic_notes.copy()

In [12]:
admissions_df.ADMISSION_TYPE.value_counts()

ADMISSION_TYPE
EMERGENCY    42071
NEWBORN       7863
ELECTIVE      7706
URGENT        1336
Name: count, dtype: int64

In [13]:
adm_grownups = admissions_df[admissions_df.ADMISSION_TYPE != "NEWBORN"]
adm_grownups

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,NaN,NaN,INTERIOR MYOCARDIAL INFARCTION,0,1
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58971,58594,98800,191113,2131-03-30 21:13:00,2131-04-02 15:02:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,ENGL,NOT SPECIFIED,SINGLE,WHITE,2131-03-30 19:44:00,2131-03-30 22:41:00,TRAUMA,0,1
58972,58595,98802,101071,2151-03-05 20:00:00,2151-03-06 09:10:00,2151-03-06 09:10:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,ENGL,CATHOLIC,WIDOWED,WHITE,2151-03-05 17:23:00,2151-03-05 21:06:00,SAH,1,1
58973,58596,98805,122631,2200-09-12 07:15:00,2200-09-20 12:08:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Private,ENGL,NOT SPECIFIED,MARRIED,WHITE,NaN,NaN,RENAL CANCER/SDA,0,1
58974,58597,98813,170407,2128-11-11 02:29:00,2128-12-22 13:11:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Private,ENGL,CATHOLIC,MARRIED,WHITE,2128-11-10 23:48:00,2128-11-11 03:16:00,S/P FALL,0,0


In [14]:
notes_df

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...
...,...,...,...,...,...,...,...,...,...,...,...
2083175,2070657,31097,115637.0,2132-01-21,2132-01-21 03:27:00,2132-01-21 03:38:00,Nursing/other,Report,17581.0,NaN,NPN\n\n\n#1 Infant remains in RA with O2 sats...
2083176,2070658,31097,115637.0,2132-01-21,2132-01-21 09:50:00,2132-01-21 09:53:00,Nursing/other,Report,19211.0,NaN,"Neonatology\nDOL #5, CGA 36 weeks.\n\nCVR: Con..."
2083177,2070659,31097,115637.0,2132-01-21,2132-01-21 16:42:00,2132-01-21 16:44:00,Nursing/other,Report,20104.0,NaN,Family Meeting Note\nFamily meeting held with ...
2083178,2070660,31097,115637.0,2132-01-21,2132-01-21 18:05:00,2132-01-21 18:16:00,Nursing/other,Report,16023.0,NaN,NPN 1800\n\n\n#1 Resp: [**Known lastname 2243*...


In [15]:

notes_df = notes_df[notes_df.HADM_ID.isin(list(adm_grownups.HADM_ID))]



In [16]:


notes_df = notes_df.dropna(subset=["TEXT", "HADM_ID"])
notes_df.head(1)

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...


In [17]:

notes_df = notes_df[notes_df.CATEGORY == "Discharge summary"]
notes_df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...


In [18]:
notes_df = notes_df.sort_values(by=["CHARTDATE"])
notes_df = notes_df.drop_duplicates(subset=["TEXT"], keep="last")
notes_df.head(1)

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
43658,36983,82574,118464.0,2100-06-09,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2100-6-7**] D...


In [19]:
combined_adm_texts = notes_df.groupby('HADM_ID')['TEXT'].apply(lambda x: '\n\n'.join(x)).reset_index()
notes_df = notes_df[notes_df.DESCRIPTION == "Report"]
notes_df = notes_df[["HADM_ID", "ROW_ID", "SUBJECT_ID", "CHARTDATE"]]
notes_df = notes_df.drop_duplicates(subset=["HADM_ID"], keep="last")
notes_df = pd.merge(combined_adm_texts, notes_df, on="HADM_ID", how="inner")
notes_df.head(1)

,HADM_ID,TEXT,ROW_ID,SUBJECT_ID,CHARTDATE
0,100001.0,Admission Date: [**2117-9-11**] ...,42102,58526,2117-09-17


In [20]:
notes_df["TEXT"] = notes_df["TEXT"].str.strip()

# remove entries without admission id, subject id or text
notes_df = notes_df.dropna(subset=["HADM_ID", "SUBJECT_ID", "TEXT"])

notes_df

,HADM_ID,TEXT,ROW_ID,SUBJECT_ID,CHARTDATE
0,100001.0,Admission Date: [**2117-9-11**] ...,42102,58526,2117-09-17
1,100003.0,Admission Date: [**2150-4-17**] ...,19215,54610,2150-04-21
2,100006.0,Admission Date: [**2108-4-6**] Discharg...,8772,9895,2108-04-17
3,100007.0,Admission Date: [**2145-3-31**] ...,50238,23018,2145-04-07
4,100009.0,Admission Date: [**2162-5-16**] ...,21119,533,2162-05-21
...,...,...,...,...,...
49043,199993.0,Admission Date: [**2161-10-23**] Discha...,16616,20785,2161-11-17
49044,199994.0,Admission Date: [**2188-7-7**] Discharg...,10570,23761,2188-07-17
49045,199995.0,Admission Date: [**2137-12-11**] Discha...,21697,19412,2137-12-28
49046,199998.0,Admission Date: [**2119-2-18**] ...,33528,27200,2119-02-24


In [21]:
import pandas as pd
import re

def extract_sections(text):
    
    sections = {
        "CHIEF_COMPLAINT": r"chief complaint:\s*([\s\S]*?)(?:\n\n|\Z)",
        "PRESENT_ILLNESS": r"present illness:\s*([\s\S]*?)(?:\n\n|\Z)",
        "MEDICAL_HISTORY": r"medical history:\s*([\s\S]*?)(?:\n\n|\Z)",
        "MEDICATION_ADM": r"medications on admission:\s*([\s\S]*?)(?:\n\n|\Z)",
        "ALLERGIES": r"allergies:\s*([\s\S]*?)(?:\n\n|\Z)",
        "PHYSICAL_EXAM": r"physical exam:\s*([\s\S]*?)(?:\n\n|\Z)",
        "FAMILY_HISTORY": r"family history:\s*([\s\S]*?)(?:\n\n|\Z)",
        "SOCIAL_HISTORY": r"social history:\s*([\s\S]*?)(?:\n\n|\Z)"
    }
    
    extracted = {}
    for key, pattern in sections.items():
        match = re.search(pattern, text, re.IGNORECASE | re.DOTALL)
        if match:
            extracted[key] = match.group(1).strip()
        else:
            extracted[key] = ""
    
    return extracted




In [22]:
# Assuming you have a DataFrame called notes_df with a 'TEXT' column

notes_df['extracted'] = notes_df['TEXT'].apply(extract_sections)

# Expand the dictionary into separate columns
notes_df = pd.concat([notes_df, notes_df['extracted'].apply(pd.Series)], axis=1).drop('extracted', axis =1)


,HADM_ID,TEXT,ROW_ID,SUBJECT_ID,CHARTDATE,CHIEF_COMPLAINT,PRESENT_ILLNESS,MEDICAL_HISTORY,MEDICATION_ADM,ALLERGIES,PHYSICAL_EXAM,FAMILY_HISTORY,SOCIAL_HISTORY
0,100001.0,Admission Date: [**2117-9-11**] ...,42102,58526,2117-09-17,"nausea, vomiting",35F w/ poorly controlled Type 1 diabetes melli...,"Type 1 diabetes mellitis w/ neuropathy, nephro...",1. citalopram 20 mg Tablet Sig: One (1) Tablet...,Levaquin,"GEN: Awake, alert, and oriented\nHEENT: PERRLA...",Both parents have HTN and T2DM. Grandfather ha...,Patient lives at home in [**Location (un) **] ...
1,100003.0,Admission Date: [**2150-4-17**] ...,19215,54610,2150-04-21,coffee ground emesis,Mr. [**Known lastname 52368**] is a 59M w HepC...,HCV Cirrhosis (tx with interferon x2 with no r...,FUROSEMIDE 20mg daily\nLISINOPRIL 10 mg daily\...,Patient recorded as having No Known Allergies ...,ON ADMISSION:\nVS: T95.9 HR 71 BP 83/36 RR 11 ...,NC,"He lives alone. He is drinking alcohol, usual..."
2,100006.0,Admission Date: [**2108-4-6**] Discharg...,8772,9895,2108-04-17,Dyspnea.,This is a 48 year old African\nAmerican female...,1. Multiple myeloma diagnosed in [**2107-12-9...,,No known drug allergies.,,Family history shows father died of an\nmyocar...,"Socially, she lives with her children and\nwor..."
3,100007.0,Admission Date: [**2145-3-31**] ...,50238,23018,2145-04-07,Severe abdominal and back pain\nUnable to take...,Ms [**Known lastname **] is a 73 year old fema...,Chronic Pancreatitis\nMigraines,Ramipril 1.25 mg Capsule Sig: Two (2) Capsule ...,Penicillins / Dilantin,T: 97.9 HR: 79 BP: 153/60 RR: 22 Spo2 100% on RA,"Father: deceased, leukemia\nBrother: colon cancer","Married, lives with husband who is a retired p..."
4,100009.0,Admission Date: [**2162-5-16**] ...,21119,533,2162-05-21,Angina,60yo man with known coronary disease (AMI in [...,"CAD-(AMI [**2143-7-3**], [**Name Prefix (Prefi...",Lisinopril 20'\nAtenolol 100'\nVytorin [**10/2...,No Known Allergies / Adverse Drug Reactions,Pulse: 58 Resp: 16 O2 sat: 97%-RA\nB/P Rig...,"Father died 50yo cirrhosis, mother died 42yo MI",Race: caucasian\nLast Dental Exam:\nLives with...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49043,199993.0,Admission Date: [**2161-10-23**] Discha...,16616,20785,2161-11-17,The patient is a 60 year old male\ntransferred...,The patient is a 60 year old man\nwith a histo...,Significant for coronary artery\ndisease statu...,,No known drug allergies.,,,Positive tobacco use. Positive ETOH use.\nNo ...
49044,199994.0,Admission Date: [**2188-7-7**] Discharg...,10570,23761,2188-07-17,,The patient is a 58 year old woman\nwith multi...,1. Morbid obesity.\n2. Congestive heart fail...,,"The patient has a reported allergy to Demerol,...",,,The patient has a 70 pack year history of\ntob...
49045,199995.0,Admission Date: [**2137-12-11**] Discha...,21697,19412,2137-12-28,,[**Known firstname **] [**Known lastname 57041...,Congenital bicuspid aortic valve\nstatus post ...,,,,,
49046,199998.0,Admission Date: [**2119-2-18**] ...,33528,27200,2119-02-24,Chest pain,This is a 69 year old male in known coronary a...,Coronary Artery Disease\nPrior PCI/Stenting to...,"Aspirin 325 qd, Cardura 1 qhs, HCTZ 25 [**Hosp...",Midazolam Hcl,"Vitals: 98.2, 122/70, 54, 18, 98%RA\nGeneral: ...",Denied premature coronary artery disease,"Married, 5 children. Stop smoking in [**2096**]."


In [23]:
notes_df

,HADM_ID,TEXT,ROW_ID,SUBJECT_ID,CHARTDATE,CHIEF_COMPLAINT,PRESENT_ILLNESS,MEDICAL_HISTORY,MEDICATION_ADM,ALLERGIES,PHYSICAL_EXAM,FAMILY_HISTORY,SOCIAL_HISTORY
0,100001.0,Admission Date: [**2117-9-11**] ...,42102,58526,2117-09-17,"nausea, vomiting",35F w/ poorly controlled Type 1 diabetes melli...,"Type 1 diabetes mellitis w/ neuropathy, nephro...",1. citalopram 20 mg Tablet Sig: One (1) Tablet...,Levaquin,"GEN: Awake, alert, and oriented\nHEENT: PERRLA...",Both parents have HTN and T2DM. Grandfather ha...,Patient lives at home in [**Location (un) **] ...
1,100003.0,Admission Date: [**2150-4-17**] ...,19215,54610,2150-04-21,coffee ground emesis,Mr. [**Known lastname 52368**] is a 59M w HepC...,HCV Cirrhosis (tx with interferon x2 with no r...,FUROSEMIDE 20mg daily\nLISINOPRIL 10 mg daily\...,Patient recorded as having No Known Allergies ...,ON ADMISSION:\nVS: T95.9 HR 71 BP 83/36 RR 11 ...,NC,"He lives alone. He is drinking alcohol, usual..."
2,100006.0,Admission Date: [**2108-4-6**] Discharg...,8772,9895,2108-04-17,Dyspnea.,This is a 48 year old African\nAmerican female...,1. Multiple myeloma diagnosed in [**2107-12-9...,,No known drug allergies.,,Family history shows father died of an\nmyocar...,"Socially, she lives with her children and\nwor..."
3,100007.0,Admission Date: [**2145-3-31**] ...,50238,23018,2145-04-07,Severe abdominal and back pain\nUnable to take...,Ms [**Known lastname **] is a 73 year old fema...,Chronic Pancreatitis\nMigraines,Ramipril 1.25 mg Capsule Sig: Two (2) Capsule ...,Penicillins / Dilantin,T: 97.9 HR: 79 BP: 153/60 RR: 22 Spo2 100% on RA,"Father: deceased, leukemia\nBrother: colon cancer","Married, lives with husband who is a retired p..."
4,100009.0,Admission Date: [**2162-5-16**] ...,21119,533,2162-05-21,Angina,60yo man with known coronary disease (AMI in [...,"CAD-(AMI [**2143-7-3**], [**Name Prefix (Prefi...",Lisinopril 20'\nAtenolol 100'\nVytorin [**10/2...,No Known Allergies / Adverse Drug Reactions,Pulse: 58 Resp: 16 O2 sat: 97%-RA\nB/P Rig...,"Father died 50yo cirrhosis, mother died 42yo MI",Race: caucasian\nLast Dental Exam:\nLives with...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49043,199993.0,Admission Date: [**2161-10-23**] Discha...,16616,20785,2161-11-17,The patient is a 60 year old male\ntransferred...,The patient is a 60 year old man\nwith a histo...,Significant for coronary artery\ndisease statu...,,No known drug allergies.,,,Positive tobacco use. Positive ETOH use.\nNo ...
49044,199994.0,Admission Date: [**2188-7-7**] Discharg...,10570,23761,2188-07-17,,The patient is a 58 year old woman\nwith multi...,1. Morbid obesity.\n2. Congestive heart fail...,,"The patient has a reported allergy to Demerol,...",,,The patient has a 70 pack year history of\ntob...
49045,199995.0,Admission Date: [**2137-12-11**] Discha...,21697,19412,2137-12-28,,[**Known firstname **] [**Known lastname 57041...,Congenital bicuspid aortic valve\nstatus post ...,,,,,
49046,199998.0,Admission Date: [**2119-2-18**] ...,33528,27200,2119-02-24,Chest pain,This is a 69 year old male in known coronary a...,Coronary Artery Disease\nPrior PCI/Stenting to...,"Aspirin 325 qd, Cardura 1 qhs, HCTZ 25 [**Hosp...",Midazolam Hcl,"Vitals: 98.2, 122/70, 54, 18, 98%RA\nGeneral: ...",Denied premature coronary artery disease,"Married, 5 children. Stop smoking in [**2096**]."


In [24]:
notes_df[(notes_df.CHIEF_COMPLAINT != "") | (notes_df.PRESENT_ILLNESS != "") |
                    (notes_df.MEDICAL_HISTORY != "")]

,HADM_ID,TEXT,ROW_ID,SUBJECT_ID,CHARTDATE,CHIEF_COMPLAINT,PRESENT_ILLNESS,MEDICAL_HISTORY,MEDICATION_ADM,ALLERGIES,PHYSICAL_EXAM,FAMILY_HISTORY,SOCIAL_HISTORY
0,100001.0,Admission Date: [**2117-9-11**] ...,42102,58526,2117-09-17,"nausea, vomiting",35F w/ poorly controlled Type 1 diabetes melli...,"Type 1 diabetes mellitis w/ neuropathy, nephro...",1. citalopram 20 mg Tablet Sig: One (1) Tablet...,Levaquin,"GEN: Awake, alert, and oriented\nHEENT: PERRLA...",Both parents have HTN and T2DM. Grandfather ha...,Patient lives at home in [**Location (un) **] ...
1,100003.0,Admission Date: [**2150-4-17**] ...,19215,54610,2150-04-21,coffee ground emesis,Mr. [**Known lastname 52368**] is a 59M w HepC...,HCV Cirrhosis (tx with interferon x2 with no r...,FUROSEMIDE 20mg daily\nLISINOPRIL 10 mg daily\...,Patient recorded as having No Known Allergies ...,ON ADMISSION:\nVS: T95.9 HR 71 BP 83/36 RR 11 ...,NC,"He lives alone. He is drinking alcohol, usual..."
2,100006.0,Admission Date: [**2108-4-6**] Discharg...,8772,9895,2108-04-17,Dyspnea.,This is a 48 year old African\nAmerican female...,1. Multiple myeloma diagnosed in [**2107-12-9...,,No known drug allergies.,,Family history shows father died of an\nmyocar...,"Socially, she lives with her children and\nwor..."
3,100007.0,Admission Date: [**2145-3-31**] ...,50238,23018,2145-04-07,Severe abdominal and back pain\nUnable to take...,Ms [**Known lastname **] is a 73 year old fema...,Chronic Pancreatitis\nMigraines,Ramipril 1.25 mg Capsule Sig: Two (2) Capsule ...,Penicillins / Dilantin,T: 97.9 HR: 79 BP: 153/60 RR: 22 Spo2 100% on RA,"Father: deceased, leukemia\nBrother: colon cancer","Married, lives with husband who is a retired p..."
4,100009.0,Admission Date: [**2162-5-16**] ...,21119,533,2162-05-21,Angina,60yo man with known coronary disease (AMI in [...,"CAD-(AMI [**2143-7-3**], [**Name Prefix (Prefi...",Lisinopril 20'\nAtenolol 100'\nVytorin [**10/2...,No Known Allergies / Adverse Drug Reactions,Pulse: 58 Resp: 16 O2 sat: 97%-RA\nB/P Rig...,"Father died 50yo cirrhosis, mother died 42yo MI",Race: caucasian\nLast Dental Exam:\nLives with...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49043,199993.0,Admission Date: [**2161-10-23**] Discha...,16616,20785,2161-11-17,The patient is a 60 year old male\ntransferred...,The patient is a 60 year old man\nwith a histo...,Significant for coronary artery\ndisease statu...,,No known drug allergies.,,,Positive tobacco use. Positive ETOH use.\nNo ...
49044,199994.0,Admission Date: [**2188-7-7**] Discharg...,10570,23761,2188-07-17,,The patient is a 58 year old woman\nwith multi...,1. Morbid obesity.\n2. Congestive heart fail...,,"The patient has a reported allergy to Demerol,...",,,The patient has a 70 pack year history of\ntob...
49045,199995.0,Admission Date: [**2137-12-11**] Discha...,21697,19412,2137-12-28,,[**Known firstname **] [**Known lastname 57041...,Congenital bicuspid aortic valve\nstatus post ...,,,,,
49046,199998.0,Admission Date: [**2119-2-18**] ...,33528,27200,2119-02-24,Chest pain,This is a 69 year old male in known coronary a...,Coronary Artery Disease\nPrior PCI/Stenting to...,"Aspirin 325 qd, Cardura 1 qhs, HCTZ 25 [**Hosp...",Midazolam Hcl,"Vitals: 98.2, 122/70, 54, 18, 98%RA\nGeneral: ...",Denied premature coronary artery disease,"Married, 5 children. Stop smoking in [**2096**]."


In [25]:
notes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49048 entries, 0 to 49047
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   HADM_ID          49048 non-null  float64
 1   TEXT             49048 non-null  object 
 2   ROW_ID           49048 non-null  int64  
 3   SUBJECT_ID       49048 non-null  int64  
 4   CHARTDATE        49048 non-null  object 
 5   CHIEF_COMPLAINT  49048 non-null  object 
 6   PRESENT_ILLNESS  49048 non-null  object 
 7   MEDICAL_HISTORY  49048 non-null  object 
 8   MEDICATION_ADM   49048 non-null  object 
 9   ALLERGIES        49048 non-null  object 
 10  PHYSICAL_EXAM    49048 non-null  object 
 11  FAMILY_HISTORY   49048 non-null  object 
 12  SOCIAL_HISTORY   49048 non-null  object 
dtypes: float64(1), int64(2), object(10)
memory usage: 4.9+ MB


In [47]:
# add section headers and combine into TEXT_ADMISSION
notes_df = notes_df.assign(TEXT="CHIEF COMPLAINT: " + notes_df.CHIEF_COMPLAINT.astype(str)
                                + '\n\n' +
                                "PRESENT ILLNESS: " + notes_df.PRESENT_ILLNESS.astype(str)
                                + '\n\n' +
                                "MEDICAL HISTORY: " + notes_df.MEDICAL_HISTORY.astype(str)
                                + '\n\n' +
                                "MEDICATION ON ADMISSION: " + notes_df.MEDICATION_ADM.astype(str)
                                + '\n\n' +
                                "ALLERGIES: " + notes_df.ALLERGIES.astype(str)
                                + '\n\n' +
                                "PHYSICAL EXAM: " + notes_df.PHYSICAL_EXAM.astype(str)
                                + '\n\n' +
                                "FAMILY HISTORY: " + notes_df.FAMILY_HISTORY.astype(str)
                                + '\n\n' +
                                "SOCIAL HISTORY: " + notes_df.SOCIAL_HISTORY.astype(str))

notes_df.head(1)

,HADM_ID,TEXT,ROW_ID,SUBJECT_ID,CHARTDATE,CHIEF_COMPLAINT,PRESENT_ILLNESS,MEDICAL_HISTORY,MEDICATION_ADM,ALLERGIES,PHYSICAL_EXAM,FAMILY_HISTORY,SOCIAL_HISTORY
0,100001.0,"CHIEF COMPLAINT: nausea, vomiting\n\nPRESENT I...",42102,58526,2117-09-17,"nausea, vomiting",35F w/ poorly controlled Type 1 diabetes melli...,"Type 1 diabetes mellitis w/ neuropathy, nephro...",1. citalopram 20 mg Tablet Sig: One (1) Tablet...,Levaquin,"GEN: Awake, alert, and oriented\nHEENT: PERRLA...",Both parents have HTN and T2DM. Grandfather ha...,Patient lives at home in [**Location (un) **] ...


In [48]:
# notes_df = filter_admission_text(notes_df)
notes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49048 entries, 0 to 49047
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   HADM_ID          49048 non-null  float64
 1   TEXT             49048 non-null  object 
 2   ROW_ID           49048 non-null  int64  
 3   SUBJECT_ID       49048 non-null  int64  
 4   CHARTDATE        49048 non-null  object 
 5   CHIEF_COMPLAINT  49048 non-null  object 
 6   PRESENT_ILLNESS  49048 non-null  object 
 7   MEDICAL_HISTORY  49048 non-null  object 
 8   MEDICATION_ADM   49048 non-null  object 
 9   ALLERGIES        49048 non-null  object 
 10  PHYSICAL_EXAM    49048 non-null  object 
 11  FAMILY_HISTORY   49048 non-null  object 
 12  SOCIAL_HISTORY   49048 non-null  object 
dtypes: float64(1), int64(2), object(10)
memory usage: 4.9+ MB


In [49]:
mimic_diagnoses = mimic_diagnoses[['SUBJECT_ID', 'HADM_ID', 'ICD9_CODE']]
mimic_diagnoses

,SUBJECT_ID,HADM_ID,ICD9_CODE
0,109,172335,40301
1,109,172335,486
2,109,172335,58281
3,109,172335,5855
4,109,172335,4254
...,...,...,...
651042,97503,188195,20280
651043,97503,188195,V5869
651044,97503,188195,V1279
651045,97503,188195,5275


In [50]:
mimic_diagnoses = mimic_diagnoses.dropna(
    how='any', subset=['ICD9_CODE'], axis=0)

# create column SHORT_CODE including first 3 digits of ICD9 code
mimic_diagnoses["SHORT_CODE"] = mimic_diagnoses.ICD9_CODE.astype(str)

/tmp/ipykernel_2654293/2114961144.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mimic_diagnoses["SHORT_CODE"] = mimic_diagnoses.ICD9_CODE.astype(str)


In [ ]:
mimic_diagnoses

,SUBJECT_ID,HADM_ID,ICD9_CODE,SHORT_CODE
0,109,172335,40301,40301
1,109,172335,486,486
2,109,172335,58281,58281
3,109,172335,5855,5855
4,109,172335,4254,4254
...,...,...,...,...
651042,97503,188195,20280,20280
651043,97503,188195,V5869,V5869
651044,97503,188195,V1279,V1279
651045,97503,188195,5275,5275


In [ ]:
mimic_diagnoses.loc[
    mimic_diagnoses['SHORT_CODE'].str.startswith("V"), 'SHORT_CODE'] = mimic_diagnoses.SHORT_CODE.apply(
    lambda x: x[:4])
mimic_diagnoses.loc[
    mimic_diagnoses['SHORT_CODE'].str.startswith("E"), 'SHORT_CODE'] = mimic_diagnoses.SHORT_CODE.apply(
    lambda x: x[:4])
mimic_diagnoses.loc[(~mimic_diagnoses.SHORT_CODE.str.startswith("E")) & (
    ~mimic_diagnoses.SHORT_CODE.str.startswith("V")), 'SHORT_CODE'] = mimic_diagnoses.SHORT_CODE.apply(
    lambda x: x[:3])

In [ ]:
mimic_diagnoses

,SUBJECT_ID,HADM_ID,ICD9_CODE,SHORT_CODE
0,109,172335,40301,403
1,109,172335,486,486
2,109,172335,58281,582
3,109,172335,5855,585
4,109,172335,4254,425
...,...,...,...,...
651042,97503,188195,20280,202
651043,97503,188195,V5869,V586
651044,97503,188195,V1279,V127
651045,97503,188195,5275,527


In [ ]:
mimic_diagnoses = mimic_diagnoses.drop_duplicates(
    ["HADM_ID", "SHORT_CODE"])

# store all ICD codes for vectorization
icd9_codes = mimic_diagnoses.SHORT_CODE.unique().tolist()

grouped_codes = mimic_diagnoses.groupby(['HADM_ID', 'SUBJECT_ID'])['SHORT_CODE'].apply(
    lambda d: ",".join(d.astype(str))).reset_index()

In [ ]:
grouped_codes = grouped_codes.rename(columns={'SHORT_CODE': 'SHORT_CODES'})

# merge discharge summaries into diagnosis table
notes_diagnoses_df = pd.merge(
    grouped_codes[['HADM_ID', 'SHORT_CODES']], mimic_notes, how='inner', on='HADM_ID')

notes_diagnoses_df

,HADM_ID,SHORT_CODES,TEXT,ROW_ID,SUBJECT_ID,CHARTDATE,CHIEF_COMPLAINT,PRESENT_ILLNESS,MEDICAL_HISTORY,MEDICATION_ADM,ALLERGIES,PHYSICAL_EXAM,FAMILY_HISTORY,SOCIAL_HISTORY


In [ ]:
mimic_utils.save_mimic_split_patient_wise(notes_diagnoses_df,
                                              label_column='SHORT_CODES',
                                              save_dir='/home/ashish/llama_inference/clinical-outcome-prediction/saved_files',
                                              task_name=task_name,
                                              seed=32)

In [ ]:
def write_icd_codes_to_file(icd_codes: List[str], data_path):
    # save ICD codes in an extra file
    with open(os.path.join(data_path, "ALL_3_DIGIT_DIA_CODES.txt"), "w", encoding="utf-8") as icd_file:
        icd_file.write(" ".join(icd_codes))

In [ ]:
write_icd_codes_to_file(icd9_codes, '/home/ashish/llama_inference/clinical-outcome-prediction/saved_files')

In [34]:
!cd ..
!ls

core.zip  experiments  LICENSE	README.md  saved_files	tasks


In [36]:
import zipfile

def create_zip(zip_filename, files):
    with zipfile.ZipFile(zip_filename, 'w') as zipf:
        for file in files:
            zipf.write(file, arcname=file.split('/')[-1])



files_to_zip = ['/home/ashish/llama_inference/clinical-outcome-prediction/saved_files/ALL_3_DIGIT_DIA_CODES.txt', '/home/ashish/llama_inference/clinical-outcome-prediction/saved_files/DIA_GROUPS_3_DIGITS_adm_test.csv', '/home/ashish/llama_inference/clinical-outcome-prediction/saved_files/DIA_GROUPS_3_DIGITS_adm_train.csv', '/home/ashish/llama_inference/clinical-outcome-prediction/saved_files/DIA_GROUPS_3_DIGITS_adm_val.csv']


create_zip('core.zip', files_to_zip)